
---

# 🗣️ Build a "Talk-to-Me" Conversationalist Chatbot

<img src="../talk_talk.png" width="400" height="400"/>

---

## 🎯 What's This About?

Not all chatbots are goldfish. 🐟  
Some actually **remember** what you said 10 seconds ago!  
Welcome to the world of **conversationalist chatbots** — where bots keep the story going, one prompt at a time.

In this lab, you'll build a chatbot that:

- **Remembers your previous questions**
- **Responds with context** (so it doesn’t say "Who are you?" every two seconds)
- **Feels more like a conversation, less like yelling into the void**

---

## 🛠️ How It Works

| Part | What’s Happening |
|:---|:---|
| Message List | Every user message and bot response gets saved |
| Full History | Each new call sends **all** past messages again |
| AI Thinks | Based on the *entire conversation*, not just the latest input |

✅ The AI’s brain:  
- First, it knew nothing.  
- Now, it’s slowly piecing your weird conversation together.

---

## 🧠 Why This Matters

- **Memory makes magic.**  
Without context, your chatbot is just a parrot.  
With memory, it becomes a guide, a friend, a co-pilot.

- **Real apps** (like ChatGPT, Claude, Gemini) all work this way under the hood.  
They just *fake* long-term memory by *replaying* everything each time.

---

# 🎯 Your Goal

Build a **talk-to-me chatbot** that:

- Keeps the chat history growing  
- Responds like it knows what’s going on  
- Feels more like a conversation, less like a quiz

---

# 🏁 Final Thought

> **Today: Your chatbot remembers a few lines.  
> Tomorrow: It remembers your life story.** Muhahaha :wink:


In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# Initialize

openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [ ]:
system_message = "You are a helpful assistant"

---

# 🛠️ Gradio’s Hidden Superpower: Sending Conversation History in OpenAI Format

---

## 🎯 What’s the Big Deal?

Gradio makes it incredibly easy to **build chatbots that remember conversations**.  
But the best part?  
It **automatically stores history** in the **exact format OpenAI models expect** — no extra work needed.

✅ That means you can **pass the history directly to OpenAI**, Claude, Gemini, or any model that expects this structure:

```json
[
  {"role": "system", "content": "Set the scene for the AI."},
  {"role": "user", "content": "User says something."},
  {"role": "assistant", "content": "AI responds."},
  {"role": "user", "content": "User says something else."},
  ...
]
```

---

## 🧠 Why Is This Awesome?

| Feature | Why It’s Great |
|:---|:---|
| Auto-history tracking | You don’t have to manually build message lists |
| OpenAI-ready format | Works instantly with GPT, Claude, Gemini APIs |
| Clean & scalable | Supports long, multi-turn conversations easily |
| Simple inputs | Gradio automatically sends full history back to your `fn` function |

---

## 🔥 How It Works Under the Hood

- Gradio **remembers** all user messages and assistant responses.
- Each time the user sends a new message, **Gradio passes the *full* conversation history** into your Python function.
- All you need to do is **accept `messages` as an input**, and you get the ready-to-send list!

✅ No history stitching.  
✅ No manual appending.  
✅ No headaches.

---

# 🎯 Final Thought

> **With Gradio, you’re always one line away from a production-ready conversational AI.**

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

## And then enter Gradio's magic!

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
system_message = "You are a helpful assistant in a AutoBody shop. You should try to gently encourage \
the customer to try items that are on sale. wiper fluid and tires are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of wiper fluid - including several that are part of our sales event.'\
Encourage the customer to buy tires if they are unsure what to get."

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
system_message += "\nIf the customer asks for brake shoes, you should respond that break shoes are not on sale today, \
but remind the customer to look at tires and wiper fluid!"

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The shope does not sell engine oil; if you are asked for engine oil, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()


---

# 🍦 Lab: The Ice Cream Chatbot (and Licorice Conspiracy)

---

Build a chatbot for an ice cream store that politely but relentlessly reminds users that **licorice ice cream is 90% off**. When a user asks about flavors, the bot should list them normally (chocolate, vanilla, strawberry, etc.) — but always slip in a little "P.S. Licorice is basically free today!" somewhere in the response. Keep the bot friendly, helpful, and mildly obsessed with getting rid of licorice.

Think about how you'd sneak this into **every answer** without sounding like a broken record — and how a real conversational assistant can balance **information + subtle persuasion**. Bonus points if the bot gets *even more desperate* if the user keeps ignoring the deal!

---


---

# 🏆 🎮 Achievement Unlocked: "Keeper of Context" Badge

```
──────────────────────────────────────────────
🎯 Mission Complete: Built a Conversationalist Bot!

🧠 New Powers Gained:
- Memory unlocked: No more goldfish-mode chats
- Context awareness: Your bot actually knows what’s going on
- Real conversation flow: Less "Who dis?", more "Let's continue!"

🏅 New Title: Context Commander

🚀 Next Quest: Handle *long* conversations without losing your mind (or your tokens)
──────────────────────────────────────────────
```